■ 사용된 PLUS OBJECT:

  - CpSysDib.CssStgList : 전략 리스트 조회 (예제 또는 사용자 전략 선택 가능)

  - CpSysDib.CssStgFind : 특정 전략조건에 해당하는 종목 리스트 조회 

In [ ]:
import sys
from PyQt5.QtWidgets import *
import win32com.client
import pandas as pd
import os

import ChatBotModel

In [ ]:
# 텔레그램 봇 생성하기
BUSbot = ChatBotModel.Bot2ndBUS()

g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpStockCode')  # 주식코드조회작업을함
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')  # 각종상태를확인할수있음
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')  # 주문오브젝트를사용하기위해필요한초기화과정

 
# Cp8537 : 종목검색 전략 조회
class Cp8537:
    def __init__(self):
        self.objRq = None
        return
 
    def requestList(self, caller):
        caller.data8537 = {}
        self.objRq = None
        self.objRq = win32com.client.Dispatch("CpSysDib.CssStgList")
 
        # 예제 전략에서 전략 리스트를 가져옵니다.
        self.objRq.SetInputValue(0, ord('1'))   # '0' : 예제전략, '1': 나의전략
        self.objRq.BlockRequest()
 
        # 통신 및 통신 에러 처리
        rqStatus = self.objRq.GetDibStatus()
        if rqStatus != 0:
            rqRet = self.objRq.GetDibMsg1()
            print("통신상태", rqStatus, rqRet)
            return False
        

        cnt = self.objRq.GetHeaderValue(0) # 0 - (long) 전략 목록 수
        flag = self.objRq.GetHeaderValue(1) # 1 - (char) 요청구분
        # print('종목검색 전략수:', cnt)

 
        for i in range(cnt):
            item = {}
            item['전략명'] = self.objRq.GetDataValue(0, i)
            item['ID'] = self.objRq.GetDataValue(1, i)
            item['전략등록일시'] = self.objRq.GetDataValue(2, i)
            item['작성자필명'] = self.objRq.GetDataValue(3, i)
            item['평균종목수'] = self.objRq.GetDataValue(4, i)
            item['평균승률'] = self.objRq.GetDataValue(5, i)
            item['평균수익'] = self.objRq.GetDataValue(6, i)
            caller.data8537[item['전략명']] = item
            # print(item)
            
        return True
 
    def requestStgID(self, id, caller):
        caller.dataStg = []
        self.objRq = None
        self.objRq = win32com.client.Dispatch("CpSysDib.CssStgFind")
        self.objRq.SetInputValue(0, id) # 전략 id 요청
        self.objRq.BlockRequest()
        # 통신 및 통신 에러 처리
        rqStatus = self.objRq.GetDibStatus()
        if rqStatus != 0:
            rqRet = self.objRq.GetDibMsg1()
            print("통신상태", rqStatus, rqRet)
            return False
 
        cnt = self.objRq.GetHeaderValue(0)  # 0 - (long) 검색된 결과 종목 수
        totcnt = self.objRq.GetHeaderValue(1)  # 1 - (long) 총 검색 종목 수
        stime = self.objRq.GetHeaderValue(2)  # 2 - (string) 검색시간
        # print('검색된 종목수:', cnt, '전체종목수:', totcnt, '검색시간:', stime)
 
        for i in range(cnt):
            item = {}
            item['code'] = self.objRq.GetDataValue(0, i)
            item['종목명'] = g_objCodeMgr.CodeToName(item['code'])
            caller.dataStg.append(item)
 
        return True
 
class MyWindow(QMainWindow):
    def __init__(self):
        #텔레그램봇 메세지 보내기
        try:
            BUSbot.sendMessage("유저전략 검색을 시작합니다.")
        except Exception as ex:
            print ("텔레그램 오류가 발생했습니다.", ex)
            pass  # telegram 처음 첫 시작시 timeout error가 발생하는 경우있어, 발생시 무시고하고 진행함
        
        super().__init__()

        self.obj8537 = Cp8537()
        self.data8537 = {}  # 전략 변수
        self.dataStg = []  # 전략의 종목 변수
 
        # 전략리스트조회
        self.obj8537.requestList(self)  # data8537에 저장됨
     
        # 전략명 지정하여 검색된 종목
        stgName = ["볼린저밴드 I","볼린저밴드 II","볼린저밴드 III"]
        for i in range(len(stgName)):
            item = self.data8537[stgName[i]]
            id = item['ID']
            name = item['전략명']

            self.obj8537.requestStgID(id, self)
            

            print('검색전략:', id, '전략명:', name, '검색종목수:', len(self.dataStg))
            for item in self.dataStg:
                print(item)
            
            #텔레그램 봇으로 메세지보내기
            BUSbot.sendMessage(name)
            for item in self.dataStg:
                BUSbot.sendMessage(item['code']+" : "+item['종목명'])

if __name__ == "__main__":
    app = QApplication(sys.argv)
    myWindow = MyWindow()
    myWindow.show()
    app.exec_()